In [17]:
import pandas as pd
import matplotlib as mp
import numpy as np
import datetime
import pprint
from math import sqrt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'
group=['ref_hash','window_nr']

---
## Read de Features

In [18]:
features = pd.read_csv('training_set6.csv',low_memory=False).set_index(group)

In [19]:
features.head()

secs_to_next_mean  secs_since_last_arrival  \
ref_hash       window_nr                                               
40621409780134 2                     7894.0                      0.0   
               2                     7894.0                      0.0   
41863526108385 1                      935.0                  70206.0   
               1                      935.0                  70206.0   
               1                      935.0                  70206.0   

                          auctions_total  auctions_last_hour  \
ref_hash       window_nr                                       
40621409780134 2                     0.0                 0.0   
               2                     0.0                 0.0   
41863526108385 1                    34.0                 0.0   
               1                    34.0                 0.0   
               1                    34.0                 0.0   

                          amount_auctions_in_weekend  is_last_weekend  \
ref_hash       window_nr                                                
40621409780134 2                                 0.0            False   
               2                                 0.0            False   
41863526108385 1                                33.0             True   
               1                                33.0             True   
               1                                33.0             True   

                          amount_dif_src  amount_events  \
ref_hash       window_nr                                  
40621409780134 2                     0.0            9.0   
               2                     0.0            9.0   
41863526108385 1                     3.0           88.0   
               1                     3.0           88.0   
               1                     3.0           88.0   

                          secs_since_last_event   wifi  ...  last_event  \
ref_hash       window_nr                                ...               
40621409780134 2                        32279.0  False  ...       363.0   
               2                        32279.0  False  ...       363.0   
41863526108385 1                        66453.0  False  ...         1.0   
               1                        66453.0  False  ...         1.0   
               1                        66453.0  False  ...         1.0   

                          before_last_event  before_before_last_event  \
ref_hash       window_nr                                                
40621409780134 2                      364.0                       7.0   
               2                      364.0                       7.0   
41863526108385 1                        1.0                       1.0   
               1                        1.0                       1.0   
               1                        1.0                       1.0   

                          pond_M_X_ev  pond_hour_ev_x  pond_hour_ev_y  \
ref_hash       window_nr                                                
40621409780134 2                False             0.9             0.9   
               2                 True             0.9             0.9   
41863526108385 1                False             0.9             0.9   
               1                False             0.9             0.2   
               1                False             0.2             0.9   

                          target_st_max  target_st_min  target_sc_max  \
ref_hash       window_nr                                                
40621409780134 2               259200.0            0.0       259200.0   
               2               259200.0            0.0       259200.0   
41863526108385 1               259200.0            0.0       259200.0   
               1               259200.0            0.0       259200.0   
               1               259200.0            0.0       259200.0   

                          target_sc_min  
ref_hash       window_nr                 


In [20]:
features.is_last_weekend.mean()

0.10146412442511474

In [21]:
new_features = features.groupby('ref_hash').mean()

In [22]:
new_features.head()

,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,...,last_event,before_last_event,before_before_last_event,pond_M_X_ev,pond_hour_ev_x,pond_hour_ev_y,target_st_max,target_st_min,target_sc_max,target_sc_min
ref_hash,,,,,,,,,,,,,,,,,,,,,
40621409780134,7894.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,9.0,32279.000000,0.0,...,363.000000,364.000000,7.0,0.5,0.900000,0.900000,259200.0,0.0,259200.0,6496.0
41863526108385,3347.666667,46804.0,22.666667,0.0,22.0,0.666667,2.0,69.5,96824.833333,0.0,...,1.333333,1.333333,1.0,0.0,0.483333,0.483333,259200.0,0.0,259200.0,6496.0
69039685746313,8173.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,10.0,12692.000000,1.0,...,287.000000,151.000000,287.0,0.5,0.900000,0.900000,259200.0,0.0,259200.0,6496.0
90072729247980,8452.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,3.0,192523.000000,0.0,...,0.000000,0.000000,0.0,1.0,0.900000,0.900000,259200.0,0.0,259200.0,6496.0
135153013040192,114.000000,70659.0,7.000000,0.0,7.0,1.000000,1.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [23]:
targets = pd.read_csv('targets.csv').set_index(group)

In [24]:
targets.head(2)

,,target_st,target_sc
ref_hash,window_nr,,
2564673204772915246,1,63743.0,64331.0
4441121667607578179,1,74414.0,87636.0


In [25]:
targets.isnull().sum()

target_st    0
target_sc    0
dtype: int64

In [26]:
training_set = features.merge(targets, how='inner', left_index=True,right_index=True)

In [27]:
training_set.head()

secs_to_next_mean  secs_since_last_arrival  \
ref_hash       window_nr                                               
40621409780134 2                     7894.0                      0.0   
               2                     7894.0                      0.0   
41863526108385 1                      935.0                  70206.0   
               1                      935.0                  70206.0   
               1                      935.0                  70206.0   

                          auctions_total  auctions_last_hour  \
ref_hash       window_nr                                       
40621409780134 2                     0.0                 0.0   
               2                     0.0                 0.0   
41863526108385 1                    34.0                 0.0   
               1                    34.0                 0.0   
               1                    34.0                 0.0   

                          amount_auctions_in_weekend  is_last_weekend  \
ref_hash       window_nr                                                
40621409780134 2                                 0.0            False   
               2                                 0.0            False   
41863526108385 1                                33.0             True   
               1                                33.0             True   
               1                                33.0             True   

                          amount_dif_src  amount_events  \
ref_hash       window_nr                                  
40621409780134 2                     0.0            9.0   
               2                     0.0            9.0   
41863526108385 1                     3.0           88.0   
               1                     3.0           88.0   
               1                     3.0           88.0   

                          secs_since_last_event   wifi  ...  \
ref_hash       window_nr                                ...   
40621409780134 2                        32279.0  False  ...   
               2                        32279.0  False  ...   
41863526108385 1                        66453.0  False  ...   
               1                        66453.0  False  ...   
               1                        66453.0  False  ...   

                          before_before_last_event  pond_M_X_ev  \
ref_hash       window_nr                                          
40621409780134 2                               7.0        False   
               2                               7.0         True   
41863526108385 1                               1.0        False   
               1                               1.0        False   
               1                               1.0        False   

                          pond_hour_ev_x  pond_hour_ev_y  target_st_max  \
ref_hash       window_nr                                                  
40621409780134 2                     0.9             0.9       259200.0   
               2                     0.9             0.9       259200.0   
41863526108385 1                     0.9             0.9       259200.0   
               1                     0.9             0.2       259200.0   
               1                     0.2             0.9       259200.0   

                          target_st_min  target_sc_max  target_sc_min  \
ref_hash       window_nr                                                
40621409780134 2                    0.0       259200.0         6496.0   
               2                    0.0       259200.0         6496.0   
41863526108385 1                    0.0       259200.0         6496.0   
               1                    0.0       259200.0         6496.0   
               1                    0.0       259200.0         6496.0   

                          target_st  target_sc  
ref_hash       window_nr                        
40621409780134 2           259200.0   259200.0  
               2           259200.0   259200.0  


In [28]:
x_st = training_set[[x for x in training_set.columns if x not in targets.columns]]
y_st = training_set['target_st']

x_sc = training_set[[x for x in training_set.columns if x not in targets.columns]]
y_sc = training_set['target_sc']

---
## Training 

In [29]:
xtrain_st, xtest_st, ytrain_st, ytest_st = train_test_split(x_st, y_st, train_size=0.7)
xtrain_sc, xtest_sc, ytrain_sc, ytest_sc = train_test_split(x_sc, y_sc, train_size=0.7)

In [30]:
model_st = xgb.XGBRegressor(objective='reg:squarederror')
model_sc = xgb.XGBRegressor(objective='reg:squarederror')

model_st.fit(xtrain_st, ytrain_st)
model_sc.fit(xtrain_sc, ytrain_sc)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [31]:
#def xgb_model_select(X, y):  
#    print ('Select Model...')
#    start_time  = datetime.datetime.now()
#    xgb_clf = xgb.XGBRegressor() 
#    parameters = {'objective':['reg:squarederror'],'n_estimators': [i for i in range(20,110,30)], 'max_depth':[i for i in range (3,6)]}
#    grid_search = GridSearchCV(estimator=xgb_clf, param_grid=parameters, cv=10, n_jobs=4, scoring='neg_mean_squared_error')
#    print("parameters:")
#    pprint.pprint(parameters)
#    grid_search.fit(X, y)
#    print("Best score: %0.3f" % grid_search.best_score_)
#    print("Best parameters set:")
#    best_parameters=grid_search.best_estimator_.get_params()
#    for param_name in sorted(parameters.keys()):
#        print("\t%s: %r" % (param_name, best_parameters[param_name]))
#    end_time = datetime.datetime.now()
#    print ('Select Done..., Time Cost: %d' % ((end_time - start_time).seconds) )

In [32]:
#xgb_model_select(x_sc,y_sc)

In [33]:
ypred_st = model_st.predict(xtest_st)
print("Error St.:", sqrt(mean_squared_error(ytest_st,ypred_st)))

ypred_sc = model_sc.predict(xtest_sc)
print("Error Sc.:", sqrt(mean_squared_error(ytest_sc,ypred_sc)))

#1.Feature default:
#Error St.: 83474.15593257314
#Error Sc.: 50173.18506919942
#2.Feature2
#('Error St.:', 82811.50353336304)
#('Error Sc.:', 48090.430099387835)
#3.Features3
#('Error St.:', 82312.86628044248)
#('Error Sc.:', 46461.809478466035)

('Error St.:', 82312.86628044248)
('Error Sc.:', 46461.809478466035)


In [34]:
model_st.feature_importances_

array([0.04301444, 0.12174844, 0.5770106 , 0.00620894, 0.18712403,
       0.00167563, 0.00371187, 0.00686975, 0.00429078, 0.00373968,
       0.00094684, 0.00668058, 0.00104673, 0.00196538, 0.01733851,
       0.01269558, 0.00393222, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ], dtype=float32)

In [35]:
model_sc.feature_importances_

array([0.02243216, 0.03796356, 0.03665763, 0.01312091, 0.0326433 ,
       0.03894129, 0.04102771, 0.03747857, 0.45700017, 0.02028239,
       0.03996604, 0.02033779, 0.03238731, 0.0688166 , 0.03140574,
       0.02990722, 0.03963157, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ], dtype=float32)

---
## Prediccion

In [36]:
# Labels a submitir con las predicciones

to_predict = pd.read_csv(root_dir + 'target_competencia_ids.csv',usecols=['ref_hash'])
to_predict.head()

,ref_hash
0,1000169251625791246_sc
1,1000169251625791246_st
2,1000395625957344683_sc
3,1000395625957344683_st
4,1003027494996471685_sc


In [37]:
X = new_features

In [38]:
y_st = model_st.predict(X)
y_sc = model_sc.predict(X)

In [39]:
results_st = X.reset_index()['ref_hash'].transform(lambda x: str(x) + '_st').rename('ref_hash').to_frame()
results_st['obj'] = y_st

results_sc = X.reset_index()['ref_hash'].transform(lambda x: str(x) + '_sc').rename('ref_hash').to_frame()
results_sc['obj'] = y_sc

results = results_st.append(results_sc)

In [40]:
targets_mean = targets.reset_index()
targets_mean = targets_mean.loc[~targets_mean['ref_hash'].isin(X.reset_index()['ref_hash'])].groupby('ref_hash')[['target_st','target_sc']].mean().reset_index()

t_st = targets_mean['ref_hash'].transform(lambda x: str(x) + '_st').rename('ref_hash').to_frame()
t_st['obj'] = targets_mean['target_st']

t_sc = targets_mean['ref_hash'].transform(lambda x: str(x) + '_sc').rename('ref_hash').to_frame()
t_sc['obj'] = targets_mean['target_sc']

r = t_st.append(t_sc)

In [41]:
results = results.append(r)

In [42]:
to_predict.merge(results, on='ref_hash', how='left')['obj'].count()

8056

In [43]:
#Armado de File Final
max_secs = 3*24*60*60
to_predict.merge(results, on='ref_hash', how='left').fillna(max_secs).to_csv('predictions_xgb6.csv', header=True, index=False)

In [44]:
to_predict.count()

ref_hash    8074
dtype: int64